### 1. 생활법령 백문백답 크롤링
- 소비자 외 일부 주제 (창업, 정보통신/기술) 추가
- 최종 결과 중 전자상거래와 무관한 주제는 수기 삭제 하였음 (총 100개 크롤링 -> 최종 데이터 86개)

In [24]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.easylaw.go.kr/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# 소비자 주제 자동 크롤링
def get_subcategories(main_id, main_name):
    url = f"{BASE_URL}CSP/OnhunqueansLstRetrieve.laf?onhunqnaAstSeq={main_id}"
    res = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")

    sub_cats = []
    for a in soup.select("a[href*='onhunqueAstSeq=']"):
        href = a["href"]
        if f"onhunqnaAstSeq={main_id}" not in href:
            continue
        sub_id = href.split("onhunqueAstSeq=")[-1].split("&")[0]
        sub_name = a.get("title", "").strip()
        sub_cats.append({
            "main_name": main_name,
            "main_id": main_id,
            "sub_name": sub_name,
            "sub_id": int(sub_id)
        })
    return sub_cats

# 상세페이지에서 전체 답변 가져오기 (줄바꿈 포함)
def fetch_full_answer_from_detail_page(ast_seq, qna_seq):
    detail_url = (
        f"{BASE_URL}CSP/OnhunqueansInfoRetrieve.laf"
        f"?onhunqnaAstSeq={ast_seq}&onhunqueSeq={qna_seq}"
    )
    res = requests.get(detail_url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")

    question_tag = soup.select_one("div.ttl")
    question_text = question_tag.get_text(strip=True) if question_tag else ""

    ans_div = soup.select_one("div.ans")
    if not ans_div:
        return question_text, ""

    # 공유 버튼 제거
    for btn in ans_div.find_all("button"):
        class_list = set(btn.get("class", []))
        if class_list & {"facebook", "twitter", "kakaoTalk"}:
            btn.decompose()

    for div in ans_div.select(".sns_pop, .recBtn, .sns"):
        div.decompose()

    # 텍스트 추출
    answer_lines = []
    for div in ans_div.find_all("div"):
        text = div.get_text(strip=True)
        if text:
            answer_lines.append(text)

    answer_text = "\n".join(answer_lines)

    return question_text, answer_text

In [25]:
get_subcategories(main_id=88, main_name="소비자")

[{'main_name': '소비자', 'main_id': 88, 'sub_name': '가공식품(농축수산물)', 'sub_id': 296},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '건강기능식품', 'sub_id': 295},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '농축수산물 소비자', 'sub_id': 300},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '다단계판매', 'sub_id': 47},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '먹는물', 'sub_id': 375},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '소비자 안전정보', 'sub_id': 401},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '소비자분쟁해결', 'sub_id': 193},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '온라인 직거래 피해', 'sub_id': 439},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '인터넷 쇼핑', 'sub_id': 258},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '택배', 'sub_id': 189},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '해외직구', 'sub_id': 383},
 {'main_name': '소비자', 'main_id': 88, 'sub_name': '화장품', 'sub_id': 341}]

In [ ]:
import json
from datetime import datetime
from collections import defaultdict
from urllib.parse import urljoin, parse_qs, urlparse


BASE_URL = "https://www.easylaw.go.kr/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

add_category_list = [
    {"main_name": "창업","main_id": 91,"sub_name": "인터넷쇼핑몰 창업자","sub_id": 104,},
    {"main_name": "정보통신/기술","main_id": 94,"sub_name": "분야별 개인정보보호","sub_id": 419,},
]

# 카테고리 자동 + 수동 조합
all_categories = []
# 소비자 주제 자동 크롤링
all_categories += get_subcategories(main_id=88, main_name="소비자")
# 수동 지정 주제 추가
all_categories += add_category_list

results = []

for cat in all_categories:
    qna_url = (
        f"{BASE_URL}CSP/OnhunqueansLstRetrieve.laf"
        f"?onhunqnaAstSeq={cat['main_id']}&onhunqueAstSeq={cat['sub_id']}"
    )
    res = requests.get(qna_url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")
    qa_list = soup.select("ul.question li.qa")  

    for qa in qa_list:
        question_tag = qa.select_one("div.ttl a")
        if not question_tag:
            continue
        href = question_tag.get("href")
        qs = parse_qs(urlparse(href).query)
        ast_seq = qs.get("onhunqnaAstSeq", [cat["main_id"]])[0]
        qna_seq = qs.get("onhunqueSeq", [None])[0]

        try:
            question_text, answer_text = fetch_full_answer_from_detail_page(ast_seq, qna_seq)
        except Exception as e:
            question_text, answer_text = "", f"(답변 수집 실패: {e})"

        results.append({
            "category": f"{cat['main_name']} > {cat['sub_name']}",
            "question": question_text,
            "answer": answer_text,
            "url": f"{BASE_URL}CSP/OnhunqueansInfoRetrieve.laf?onhunqnaAstSeq={ast_seq}&onhunqueSeq={qna_seq}",
            "source": "생활법령정보 백문백답",
            "crawl_date": datetime.today().strftime('%Y-%m-%d')
        })

# 저장
with open("law_qa_list.jsonl", "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# 카테고리별 개수 집계
category_counts = defaultdict(int)
for item in results:
    category_counts[item["category"]] += 1

print("카테고리별 크롤링 결과:")
for cat, count in sorted(category_counts.items()):
    print(f" - {cat}: {count}건")

print(f"\n >> 총 {len(results)}건 저장됨 (law_qa_list.jsonl)")


카테고리별 크롤링 결과:
 - 소비자 > 가공식품(농축수산물): 8건
 - 소비자 > 건강기능식품: 6건
 - 소비자 > 농축수산물 소비자: 5건
 - 소비자 > 다단계판매: 2건
 - 소비자 > 먹는물: 5건
 - 소비자 > 소비자 안전정보: 10건
 - 소비자 > 소비자분쟁해결: 6건
 - 소비자 > 온라인 직거래 피해: 10건
 - 소비자 > 인터넷 쇼핑: 8건
 - 소비자 > 택배: 10건
 - 소비자 > 해외직구: 6건
 - 소비자 > 화장품: 5건
 - 정보통신/기술 > 분야별 개인정보보호: 10건
 - 창업 > 인터넷쇼핑몰 창업자: 9건

 >> 총 100건 저장됨 (law_qa_selected_categories.jsonl)


### [Sample] JSONL → Document 리스트 변환

In [27]:
from langchain.schema import Document
import json

def load_jsonl_to_documents(
    filepath,
    question_field="question",
    answer_field="answer",
    category_field="category",
    source_field="source",
    url_field="url",
    date_field="crawl_date",
    doc_type="질의응답"
):
    documents = []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)

            question = data.get(question_field, "").strip()
            answer = data.get(answer_field, "").strip()
            category = data.get(category_field, "기타")
            source = data.get(source_field, "")
            url = data.get(url_field, "")
            crawl_date = data.get(date_field, "")

            content = f"""[백문백답]\n다음은 {category} 분야의 질의응답입니다.

[Q&A]
Q: {question}
A: {answer}
"""

            doc = Document(
                page_content=content,
                metadata={
                    "category": category,
                    "source": source,
                    "type": doc_type,
                    "url": url,
                    "crawl_date": crawl_date
                }
            )

            documents.append(doc)

    return documents

In [ ]:
docs = load_jsonl_to_documents(
    filepath="law_qa_list.jsonl",
    question_field="question",
    answer_field="answer",
    category_field="category"
)

print(f">> 총 {len(docs)}개의 Document 로드됨")
print(docs[0].page_content)
print(docs[0].metadata)


>> 총 100개의 Document 로드됨
[백문백답]
다음은 소비자 > 가공식품(농축수산물) 분야의 질의응답입니다.

[Q&A]
Q: 제가 초콜릿을 구입했는데 포장지에 각종 영양성분들이 표시되어 있는 것을 보았습니다. 초콜릿 외에 영양성분표시기준의 대상이 되는 식품에는 어떤 것들이 있나요?
A: 초콜릿류 외에도 과자류 중 과자, 캔디류 및 빙과류, 빵류 및 만두류 등 다음에 해당하는 식품에는 영양표시를 해야 합니다.
◇영양표시 대상식품
☞ 영양표시 대상식품은 다음과 같습니다.
영양표시 대상식품레토르트식품(조리가공한 식품을 특수한 주머니에 넣어 밀봉한 후 고열로 가열 살균한 가공식품을 말하며, 축산물은 제외)특수의료용도식품과자류, 빵류 또는 떡류장류(한식메주를 이용한 한식간장은 제외)빙과류조미식품(발효식초, 소스류, 카레 및 향신료조제품)코코아 가공품류 또는 초콜릿류절임류 또는 조림류(김치류 중 배추김치만 해당하고, 절임식품 중 절임배추는 제외)당류농산가공식품류잼류식육가공품두부류 또는 묵류알가공품류(알 내용물 100% 제품은 제외)식용유지류(油脂類)(모조치즈 및 기타 식용유지가공품은 제외)유가공품면류수산가공식품류(수산물 100% 제품은 제외)음료류(다류 중 침출자·고형차와 커피 중 볶은커피 및 인스턴트 커피는 제외)즉석식품류특수영양식품건강기능식품

{'category': '소비자 > 가공식품(농축수산물)', 'source': '생활법령정보 백문백답', 'type': '질의응답', 'url': 'https://www.easylaw.go.kr/CSP/OnhunqueansInfoRetrieve.laf?onhunqnaAstSeq=88&onhunqueSeq=4972', 'crawl_date': '2025-06-06'}
